# Fine-Tuning ResNet50 on iNaturalist Dataset with Train/Val Split and Evaluation

In [9]:

import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split
import torch.optim as optim
from torch.optim import lr_scheduler
import matplotlib.pyplot as plt
import numpy as np
import os


In [10]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [ ]:

dataset = datasets.ImageFolder(root=r'C:\Users\alokg\Desktop\A_2\nature_12K\inaturalist_12K\train', transform=transform)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)


In [12]:

model = models.resnet50(pretrained=True)

# Freeze all layers first
for param in model.parameters():
    param.requires_grad = False

# Unfreeze last block and final FC layer
for param in model.layer4.parameters():
    param.requires_grad = True
for param in model.fc.parameters():
    param.requires_grad = True

# Modify final FC layer
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)

model = model.to(device)


In [13]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-4)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)


In [14]:

num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    train_loss, train_corrects = 0.0, 0
    total = 0
    
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        _, preds = torch.max(outputs, 1)
        train_loss += loss.item() * inputs.size(0)
        train_corrects += torch.sum(preds == labels.data)
        total += labels.size(0)
    
    train_loss /= total
    train_acc = train_corrects.double() / total
    
    model.eval()
    val_loss, val_corrects = 0.0, 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            _, preds = torch.max(outputs, 1)
            val_loss += loss.item() * inputs.size(0)
            val_corrects += torch.sum(preds == labels.data)
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_corrects.double() / val_total

    print(f'Epoch {epoch+1}/{num_epochs} '
          f'Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}, '
          f'Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}')
    
    scheduler.step()

# Save model
torch.save(model.state_dict(), 'fine_tuned_resnet50.pth')


Epoch 1/5 Train Loss: 1.3416, Train Acc: 0.5813, Val Loss: 0.7408, Val Acc: 0.7650
Epoch 2/5 Train Loss: 0.4750, Train Acc: 0.8619, Val Loss: 0.7245, Val Acc: 0.7525
Epoch 3/5 Train Loss: 0.1604, Train Acc: 0.9719, Val Loss: 0.6973, Val Acc: 0.7750
Epoch 4/5 Train Loss: 0.0679, Train Acc: 0.9881, Val Loss: 0.6940, Val Acc: 0.7850
Epoch 5/5 Train Loss: 0.0334, Train Acc: 0.9981, Val Loss: 0.7083, Val Acc: 0.7875


In [15]:

# Load test dataset
test_dataset = datasets.ImageFolder(root=r'C:\Users\alokg\Desktop\A_2\nature_12K\inaturalist_12K\val', transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load model and set to eval
model.load_state_dict(torch.load('fine_tuned_resnet50.pth', map_location=device))
model.to(device)
model.eval()

# Collect correct and incorrect predictions
correct_samples = []
incorrect_samples = []

# Evaluate overall test accuracy
correct = 0
total = 0
model.eval()

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

test_acc = 100 * correct / total
print(f"Test Accuracy: {test_acc:.2f}%")


Test Accuracy: 95.75%
